In [1]:
import Base: push!, pop!, isempty

mutable struct BinaryMinHeap{T}
    data::Vector{T}
    BinaryMinHeap{T}() where T = new(T[])
end

@inline function Base.push!(heap::BinaryMinHeap{T}, value::T) where T
    data = heap.data
    len = length(data) + 1
    resize!(data, len)
    _siftup!(data, len, value)
    heap
end

@inline function Base.pop!(heap::BinaryMinHeap{T}) where T
    data = heap.data
    lastelt = pop!(data)
    if !isempty(data)
        returnitem = @inbounds data[1]
        _siftdown!(data, 1, lastelt)
        returnitem
    else
        lastelt
    end
end

@inline Base.isempty(heap::BinaryMinHeap) = isempty(heap.data)

@inline function _siftup!(data::Vector{T}, mut_pos::Int, value::T) where T
    @inbounds while mut_pos > 1
        parentpos = mut_pos >>> 1
        parent = data[parentpos]
        if value < parent
            data[mut_pos] = parent
            mut_pos = parentpos
        else
            break
        end
    end
    @inbounds data[mut_pos] = value
end

@inline function _siftdown!(data::Vector{T}, mut_pos::Int, value::T) where T
    len = length(data)
    half = len >>> 1
    @inbounds while mut_pos <= half
        childpos = mut_pos << 1
        child = data[childpos]
        rightpos = childpos + 1
        if rightpos <= len && data[rightpos] < child
            childpos = rightpos
            child = data[rightpos]
        end
        if child < value
            data[mut_pos] = child
            mut_pos = childpos
        else
            break
        end
    end
    @inbounds data[mut_pos] = value
end


_siftdown! (generic function with 1 method)

In [2]:
using DataStructures
using BenchmarkTools
using Random

# Benchmark functions
function bench_push!(heap, values)
    for v in values
        push!(heap, v)
    end
end

function bench_pop!(heap, n)
    for _ in 1:n
        pop!(heap)
    end
end

function run_benchmarks(n)
    println("Running benchmarks with $n elements")
    
    # Generate random data
    data = rand(Int, n)
    
    # Custom BinaryMinHeap
    custom_heap = BinaryMinHeap{Int}()
    custom_push = @benchmark bench_push!($custom_heap, $data) samples=30 evals=3
    custom_heap = BinaryMinHeap{Int}()
    bench_push!(custom_heap, data)
    custom_pop = @benchmark begin
        bench_push!($custom_heap, $data)
        bench_pop!($custom_heap, $n)
    end samples=30 evals=3
    
    # DataStructures.jl BinaryMinHeap
    ds_heap = DataStructures.BinaryMinHeap{Int}()
    bm_push = @benchmark bench_push!($ds_heap, $data) samples=30 evals=3
    ds_heap = DataStructures.BinaryMinHeap{Int}()
    bm_pop = @benchmark begin
        bench_push!($ds_heap, $data)
        bench_pop!($ds_heap, $n)
    end samples=30 evals=3
    
    println("Custom          Push: ", median(custom_push))
    println("DataStructures  Push: ", median(bm_push))
    println("Custom          Pop:  ", median(custom_pop))
    println("DataStructures  Pop:  ", median(bm_pop))
    println()
end

# Run benchmarks for different sizes
for n in [1_000, 300_000] #, 1_000_000]
    run_benchmarks(n)
end

Running benchmarks with 1000 elements


Custom          Push: TrialEstimate(12.102 μs)
DataStructures  Push: TrialEstimate(10.135 μs)
Custom          Pop:  TrialEstimate(24.002 μs)
DataStructures  Pop:  TrialEstimate(18.501 μs)

Running benchmarks with 300000 elements
Custom          Push: TrialEstimate(4.009 ms)
DataStructures  Push: TrialEstimate(3.888 ms)
Custom          Pop:  TrialEstimate(22.775 ms)
DataStructures  Pop:  TrialEstimate(22.559 ms)

